# English Character classifier
###### Charlie Rosander, 2023-08

In this project we will create a CNN to recognise and classify english, handwritten characters (0-9, A-Z).

The dataset we are using is the "English handwritten characters" from kaggle: 
https://www.kaggle.com/datasets/dhruvildave/english-handwritten-characters-dataset

"This dataset contains 3,410 images of handwritten characters in English. This is a classification dataset that can be used for Computer Vision tasks. It contains 62 classes with 55 images of each class. The 62 classes are 0-9, A-Z and a-z."

# The dataset
Looking over the dataset i can see right away that all of the pictures are written in a program like paint and not by hand with pen and paper, which is a bit of a shame as this will most likely overfit the model and make the model perform worse on real handwritten characters.

We will see how it performs and later on I will do a "real-world" test with my own handwriting to see if I can test the limits of the model.

# Procedure

We will start with the imports and then load the data into a dataframe. We will then do some preprocessing and then create the model, followed by the testing and analysis of the model. I will make the model savable so that we can continue to train it later on if we want to.

In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1, l2, l1_l2
from sklearn.model_selection import train_test_split

#### Loading the data and labeling it through indexing the filenames.
This way I dont have to seperate the images into their own directories or rename the files etc, as looking over the data I realised right away that the first 3 digits in the filename represents that characters class.

In [9]:
def load_images(directory_path):
    images = []
    labels = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".png"):
            img = cv2.imread(os.path.join(directory_path, filename))
            img = cv2.resize(img, (84, 84)) # or any size you want
            img = img / 255.0 # Normalization
            
            # Extract label from filename
            label = int(filename[3:6]) # Index 3-6 of the filename is the label
            labels.append(label)
            
            images.append(img)
        
    images = np.array(images)
    labels = to_categorical(labels)


    return images, labels

directory_path = "./EngChars/Img/"
images, labels = load_images(directory_path)

Split the data into train/validation.

In [10]:
# Split data into training and testing
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [15]:
# Kollar så att det ser ut som vi förväntar oss, vilket det gör

print(x_train.shape)
print(x_test.shape)

(2728, 84, 84, 3)
(682, 84, 84, 3)


#### Create the model
Here we create our model, I have chosen to use 4 hidden layers with 2 convolutional layers and 2 dense layers. 

In [16]:
# Create model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(84, 84, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(26, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])